##Multithreading in Python

Abbiamo visto a lezione che un processo è un programma in esecuzione, che più processi possono essere eseguiti in parallelo sullo stesso computer e che un processo a sua volta può avere thread multipli. 

Le differenze principali tra processi e thread sono:

Avvio/terminazione sono operazioni meno costose per i thread 
I thread  richiedono meno risorse dei processi
I processi non condividono la memoria mentre i thread condividono la stessa memoria virtuale e quindi richiedono particolari operazioni di sincronizzazione per evitare conflitti in lettura/scrittura

Per quanto detto sopra il multithreading seppur potenzialmente pi efficiente grazie all'introduzione del parallelismo nell'esecuzione è  una tecnica di programmazione difficile da gestire. Questo  è particolarmente vero nel contesto di Python a causa del GIL, il lock globale dell'interprete.

Il lock globale dell'interprete (GIL) è uno degli argomenti più controversi di Python. In CPython, l'implementazione più popolare di Python, il GIL è un mutex che forza l'esecuzione sequenziale del codice. Il GIL rende semplice l'integrazione di Python con librerie esterne che non sono thread-safe ed, in generale, rende il codice non-parallelo più veloce. Tuttavia, a causa di GIL, non possiamo realizzare il parallelismo vero tramite il multithreading. In sostanza, due thread nativi diversi dello stesso processo non possono eseguire il codice Python concorrentemente ma devono invece alternare l'esecuzione di loro blocchi di istruzioni nell'interprete (simulando quindi la concorrenza come nel multitasking).


Ci sono casi in cui è ancora possibile eseguire cose in parallelo, 
ad esempio operazioni di I/O:


Per questo motivo è possibile creare e lanciare thread in Python usando la libreria "threading": https://docs.python.org/3/library/threading.html#condition-objects

Vediamo qualche esempio che poi riprenderemo in linguaggi come C e Java.


Il seguente programma è un semplice programma di calcolo (CPU-bound) sequenziale

In [ ]:
#CPU-bound sequential Python program 

COUNT = 50000000

def countdown(n): 
  while n>0: n -= 1

start = time.time()
countdown(COUNT)
end = time.time()

print('Time taken in seconds -', end - start)

Time taken in seconds - 3.7248575687408447


Importiamo ora il modulo "threading" per creare due flussi di esecuzione spezzando in due parti parallele il calcolo.

In [ ]:
#CPU-bound parallel Python program 
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
  while n>0: n -= 1

t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print('Time taken in seconds -', end - start)

Time taken in seconds - 3.742682695388794


A causa del GIL, l'interprete non riesce veramente a parallelizzare il calcolo ma è costretto ad alternare l'esecuzione delle istruzioni con effetto di appensantire invece che migliorare l'efficienza. Questo non è quindi un uso corretto del modulo threading di Python.


La rimozione del GIL avrebbe reso Python 3 più lento rispetto a Python 2  single-thread.
Tuttavia Python 3 ha apportato un notevole miglioramento al GIL della versione 2. Infatti Python 3 forza i thread a rilasciare il GIL dopo un intervallo fisso di utilizzo continuo della CPU.
Inoltre è stato aggiunto un meccanismo per esaminare il numero di richieste di acquisizione del GIL da parte dei vari thread per evitare che sempre uno stesso thread mantenga il controllo della CPU (scheduler).
Non tutti gli interpreti Python usano GIL. Ad esempio Jython, IronPython and PyPy, non includono questo meccanismo. Vediamo ora un esempio dove invece è sensato usare threading. Consideriamo una serie di operazioni di I/O ad esempio effettuare 4 richieste HTTP (anche dello stesso URL).

Vediamo sotto un possibile codice sequenziale.

In [ ]:
import requests
import time

def download_site(url, session):
    with session.get(url) as response:
        print(f"{len(response.content)}")

def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)


if __name__ == "__main__":
    sites = ["http://www.cython.org"] * 4
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

30474
30474
30474
30474
Downloaded 4 in 0.15180635452270508 seconds


Introduciamo ora dei thread per eseguire in parallelo gli accessi HTTP (operazioni I/O)

In [ ]:
import requests
import time
from threading import Thread

url = "http://www.cython.org"

t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))

def download_site(session):
  with session.get(url) as response:
    print(f"{len(response.content)}")      
      
with requests.Session() as session:
  t1=Thread(target=download_site, args=(session,))
  t2=Thread(target=download_site, args=(session,))
  t3=Thread(target=download_site, args=(session,))
  t4=Thread(target=download_site, args=(session,))
  start = time.time()
  t1.start()
  t2.start()
  t3.start()
  t4.start()
  t1.join()
  t2.join()
  t3.join()
  t4.join()
  end = time.time()
  duration = end-start
  print(f"Downloaded in {duration} seconds")

30474
30474
30474
30474
Downloaded in 0.08198165893554688 seconds


In questo caso trattandosi di operazioni di I/O che quindi non dipendono dal GIL, la loro esecuzione è veramente concorrente (e parallela nel numero di core della macchina) e quindi può portare a benefici in termini di efficienza.

Tra i vari meccanismi di comunicazione tra thread forniti da Python troviamo anche i Condition object. Sono oggetti che i thread possono usare per mettersi in attesa di notifiche da altri thread. Ad esempio un thread "master" potrebbe sincronizzare le operazioni di diversi thread slave, magari con diversa velocità, 
inviando di tanto in tanto messaggi di notifica come nel seguente esempio.

In [ ]:
import random, time
from threading import Thread, Condition

def m(condition):
  print("m")
  time.sleep(10000)
  with condition:
    condition.notifyAll() #Message to threads

def s1(condition,stop_event):
  with condition:
    condition.wait() #Wait for message
    print("s1")

def s2(condition,stop_event):
  with condition:
    condition.wait() #Wait for message
    print("s2")

condition = Condition()
th1 = Thread(name='master', target=m, args=(condition))
th2 = Thread(name='s1', target=s1, args=(condition))
th3 = Thread(name='s2', target=s2, args=(condition))

th1.start()
th2.start()
th3.start()


In [ ]:
import random, time
from threading import Thread, Condition

def m(condition):
  print("m")
  with condition:
    condition.notifyAll() #Message to threads

def s1(condition):
  with condition:
    condition.wait() #Wait for message
    print("s1")

def s2(condition):
  with condition:
    condition.wait() #Wait for message
    print("s2")

condition = Condition()
th1 = Thread(name='master', target=m, args=(condition,))
th2 = Thread(name='s1', target=s1, args=(condition,))
th3 = Thread(name='s2', target=s2, args=(condition,))

th2.start()
th3.start()
time.sleep(0.5)
th1.start()

m
s2
s1


In questo esempio i thread th2 e th3 si fermano in attesa della notifica (inviata da th1 tramite notifyall) tramite il Condition object "condition" (che va interepretato come una sorta di canale di comunicazione). Quando th1 viene lanciato dopo la stampa di "m" manda la notifica che risveglia "th2" e "th2"

Per maggiori approfondimenti sui Condition object: 
https://docs.python.org/3/library/threading.html#condition-objects